# Westeros Tutorial - Extension of GAMS Formulation

This tutorial demonstrates the procedure for changing the GAMS formulation in MESSAGEix. This will be useful for adding new sets, parameters, variables, and equations, to generate a different output or to represent new phenomena, sectors, etc.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

## Part 1: Adding a new variable `INVEST`
In the first part, we change the GAMS code by adding a new equation called `INVESTMENT_EQUIVALENCE`, which calculates the investment per region in each model period as a new variable called `INVEST`.

### 1.1. Modifying GAMS files
For representing the new equation, please do the following modifications in the GAMS file `model_core.gms`:
1. Introduce a new positive variable for "investment per node and year" as: `INVEST(node, year_all)`
2. Introduce a new equation for "investment per node and year" as: `INVESTMENT_EQUIVALENCE`
3. Specify the new equation with the following notation (please don't forget ";" at the end):

`INVESTMENT_EQUIVALENCE(node, year) ..
    INVEST(node, year) =E= SUM(tec$( map_tec(node,tec,year) ), ( inv_cost(node,tec,year) * construction_time_factor(node,tec,year) * end_of_horizon_factor(node,tec,year) * CAP_NEW(node,tec,year) )$( inv_tec(tec) )
);`

If you are not sure, you can check these changes here:

At this stage, you do not need to update the model documentation as it is done in the link above. But this is a good modeling practice for documenting the new sets, parameters, variables, and equations in the GAMS files using inline Latex notes.

### 1.2. Solving the scenario after modifications in GAMS:
We clone a "baseline" scenario and solve it again with the new changes in the GAMS side and check what the outcome is.

In [ ]:
# Loading the baseline scenario and cloning to a new scenario
model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(model, "investment", keep_solution=False)

In [ ]:
# Solving the new scenario again (testing the changes in the GAMS code)
scen.solve()

### 1.3. Checking the outcome
Let's compare the objective value of the "baseline" and "investment" scenarios. We use the python method `assert` for doing this assertion. If this equality does not hold true, the assertion method will throw an error. 

In [ ]:
# We assert that the objective values are the same
assert base.var("OBJ")["lvl"] == scen.var("OBJ")["lvl"]

In [ ]:
# We can compare the activity of one technology in one year too
assert base.var("ACT", {"technology": "coal_ppl"})["lvl"].sum() == scen.var("ACT", {"technology": "coal_ppl"})["lvl"].sum()

#### Observation 1: The results are the same
As asserted above, the objective value, i.e., the total cost of the system, as well as the level of activity of "coal_ppl" is the same in both scenarios. This means the modifications we introduced in the GAMS side have not changed the output of the model.

#### Observation 2: variable `INVEST` in the output GDX file
If we check the output GDX file of the new scenario, we should be able to see the new avriable `INVEST`. This means that the new variable has been calculated as part of the GAMS mathematical model.

Now, let's check if we can fetch the information about the new variable `INVEST`:

In [ ]:
# Reading the content variable "INVEST"
scen.var("INVEST")

#### Observation (3): Python side does not know about changes in GAMS
As the error message above states "There exists no variable 'INVEST'!". This means that the python side, i.e., `message_ix` code is not aware of the changes we have done in the GAMS side. Therefore, we cannot retrieve the content of variable `INVEST` from the GDX file, even though this variable exists there.

### 1.4. Updating the python code based on changes in GAMS
In the `message_ix` python package, there is a configuration file for specifying and initializing the model sets, parameters, and variables. This file is called [`models.py`](https://github.com/iiasa/message_ix/blob/main/message_ix/models.py).

1. For updating the file, please introduce the new variable `INVEST` with the format shown in the file, i.e., add this line under "MESSAGE_ITEMS"/"Variables" (see line 266 of the file):

`"INVEST": item("var", "n y"),`

This notation informs the `message_ix` package that there exists a variable called `INVEST` with index sets of "n" for "node" and "y" for "year". After adding this information, the python side will initiate this variable for any new or cloned scenario.

2. For being able to retrieve the content of this new variable, we need to explicitly pass it through the `var_list` option when calling `solve()`. This has to be done because this variable is not among the default variables of `message_ix`. So, the notation for solving will be:

`solve(var_list=["INVEST"])`


In [ ]:
# Now, let's solve the scenario again
scen = scen.clone(keep_solution=False)
scen.solve(var_list=["INVEST"])

Now, let's check the content of `INVEST`:

In [ ]:
scen.var("INVEST")

### Congratulations!
You were able to modify the GAMS code, add a new variable and equation, update the python code accordingly, and keep your modeling workflow working seamlessly.

## Part II: Adding a new MESSAGEix parameter 
In the second part of this tutorial, we further modify the GAMS code by adding a new parameter. The goal is to understand what modifications should be done when a new parameter is added.

### 2.1. An upper bound on investment
The new parameter will be an upper bound on investment called `bound_investment_up`. This parameter can be defined per technology types, but here for simplicity we define this bound per node and year, similar to variable `INVEST`. We can define this bound as an absolute value, i.e., maximum amount that can be invested, or we can define it relative to the total system costs, e.g., specifying that investment can be up to 80% of total costs. We choose the former.

Adding a parameter in the GAMS side is different from adding variables shown above. Because GAMS expects this parameter as an input data to be passed from the python side. Therefore, the following modifications are needed:

#### a. GAMS side

1. Define the new parameter in the GAMS file `parameter_def.gms`
2. Add the new parameter to the items to be loaded in `data_load.gms`
3. Introduce a new equation in `model_core.gms` called `INVESTMENT_CONSTRAINT`
4. Add the following equation to `model_core.gms` for the new bound:

`INVESTMENT_CONSTRAINT(node, year)$(bound_investment_up(node, year) ) ..
    INVEST(node, year) =L= bound_investment_up(node, year);`

#### Notice:
Please note that we filter this equation by `bound_investment_up` using $ sign in GAMS. This means that if this parameter is not defined in a scenario, this equation will not be in effect. This way, we retain backward compatibility, i.e., we can still solve our scenarios that do not have bound on investment.

#### b. Python side
As mentioned in Part I, we need to update the `message_ix` python code about the new changes. This can be simply done by adding a line of code to the model configuration file (`models.py`) as shown above:

5. Add new parameter to the `MESSAGE_ITEMS` in the model configuration file `models.py` as:

`"bound_investment_up": item("par", "n y"),`


In [ ]:
# Now, let's clone a new scenario and solve
scen2 = scen.clone(scenario="investment_bound", keep_solution=False)
scen2.solve(var_list=["INVEST"])

In [ ]:
df = scen2.var("INVEST", {"node": "Westeros"})
df

Everything seems fine. Now, we can add a value to `bound_investment_up` too. Let's assume an investment cap of 3000 million $ per year, and see if the model solves.

In [ ]:
# Adding input data for "bound_investment_up"
bound = pd.DataFrame({
    "node": "Westeros",
    "year": [700, 710, 720],
    "value": 3000,
    "unit": "M$",
    })
# Check the input data before adding to the scenario
bound

In [ ]:
# Adding the unit to the platform (if not exists yet)
mp.add_unit("M$")

# Adding data to the scenario
scen2.remove_solution()
scen2.check_out()
scen2.add_par("bound_investment_up", bound)
scen2.commit("bound added")

In [ ]:
# Solving the scenario
scen2.solve(var_list=["INVEST"])

### 2.2. Checking the results
In this Section, we check the results after adding the bound compared to the case before. We do this comparison on a few outputs, including:
- investment needs
- total system costs
- price of "light"

In [ ]:
# Investment needs before the bound
scen.var("INVEST", {"node": "Westeros"})

In [ ]:
# Investment needs after the bound
scen2.var("INVEST", {"node": "Westeros"})

#### Observation (1): Investment dynamics
The results show that after adding an upper bound on investment, the model has to distribute investment differently, to remain under the maximum threshold. This has resulted in higher investments in 700 and 720 compared to the previous scenario without investment bounds.

Let's compare the objective value, i.e., the total system costs

In [ ]:
# Difference in the total system costs after investment limits (M$)
scen2.var("OBJ")["lvl"] - scen.var("OBJ")["lvl"]

#### Observation (2): Total system costs
After adding an upper bound on investment, the total costs of the system has increased compared to the previous scenario without investment bounds. More importantly, the increase in total system cost is not only due to an increase in total investment but also a higher O&M costs.

**Question**: how can we calculate the change in O&M cost?

## Exercise
1- Modify the equation `INVESTMENT_EQUIVALENCE` so that the upper bound on investment will be as % of total system costs.

2- How can we relate the upper bound on investment as % of GDP in MESSAGEix?

In [ ]:
# Close the connection to the database
mp.close_db()